In [116]:
finetune_rep_dir = './finetune_rep'
prov_rep_dir = './prov_rep'

finetune_true_dir = './finetune'
prov_true_dir = './prov'

fill_na = '______'

In [117]:
# All on months = 36.
# I don't recall specifically if the code was exactly the same on both runs
# but am testing replication and don't recall making any changes to code logic (I think at most to imports)

In [118]:
from utils import split_gen, load_splits
from utils_child import child_models
import pandas as pd

import os
from os.path import join, exists

import config

import glob

In [119]:
all_args = config.childes_model_args + [('child', name) for name in child_models.get_child_names()] 

In [120]:
def compare_with_na(df1, df2):
    return df1.fillna(fill_na).equals(df2.fillna(fill_na))

Need to manually set the finetune_dir and prov_dir in config.py, then run these functions after total regeneration.

In [121]:
# Check that the text files match for all of the splits, for the tags version.
# Young and old are implicit in the checks.

def get_texts(split, name, phase, base_dir):
    
    this_split_loc = split_gen.get_split_folder(split, name, base_dir)
    with open(join(this_split_loc, f"{phase}.txt"), 'r') as f:
        return f.readlines()
        
for s, d in all_args:
    for this_phase in ['train', 'val']:
        assert get_texts(s, d, this_phase, finetune_rep_dir) == get_texts(s, d, this_phase, finetune_true_dir), f"Failed on: {s}, {d}"

print('Passed')

Passed


In [122]:
# Check that the vocabulary file (from CHILDES) also matches everything

def get_vocab(base_dir):
    this_folder = split_gen.get_split_folder('all', 'all', base_dir)
    # Drop index, leave the two data columns (word, count) -- or pd.equals will try to compare on non-data parts.
    return pd.read_csv(join(this_folder, 'chi_vocab_train.csv')).sort_values(by = ['count', 'word'], ascending = False).reset_index(drop = True)[['word', 'count']]

dfv1 = get_vocab(finetune_rep_dir)
dfv2 = get_vocab(finetune_true_dir)

assert compare_with_na(dfv1, dfv2)

print('Passed')

Passed


In [ ]:
# Check that the all_tokens_phono dataframe matches -- be careful for NaN issues here.

def get_this_phono(base_dir):
    return pd.read_pickle(join(base_dir, 'pvd_all_tokens_phono_for_eval.pkl'))
    
assert compare_with_na(get_this_phono(prov_rep_dir), get_this_phono(prov_true_dir))

print('Passed')

In [ ]:

def get_paths(folder):
    return sorted(glob.glob(join(folder, '*')))
    
base_time = 'across_time_samples'

all_rep = sorted(get_paths(join(prov_rep_dir, base_time)))
all_compare = sorted(get_paths(join(prov_true_dir, base_time)))

for p1, p2 in zip(all_rep, all_compare):
    assert pd.read_csv(p1).equals(pd.read_csv(p2)), f"Failed on path: {p1}"
    
print('Passed')

In [ ]:
# Compare loading the beta values

def get_beta_paths(base_prov_dir, arg_set = all_args):
    
    all_beta_samples = [
        pd.read_csv(join(split_gen.get_split_folder(s, d, base_prov_dir), 'success_utts_beta_5000_val.csv'))
        for s, d in arg_set
    ]
        
    return all_beta_samples

for sample1, sample2 in zip(get_beta_paths(prov_true_dir), get_beta_paths(prov_rep_dir)):
    assert sample1.equals(sample2)

print("Passed.")